# Benin Solar Farm EDA

## Overview
This notebook performs exploratory data analysis (EDA) on the Benin solar farm dataset.  
We focus on:
- Profiling and cleaning the data
- Outlier detection
- Time-series analysis
- Sensor cleaning impact
- Correlation and relationship analysis
- Wind & distribution visualization
- Temperature & humidity analysis
- Bubble chart visualization
- Summary insights and strategic recommendations


## 0. Setup — Import Libraries & Helper Functions
We import the necessary Python libraries and define helper functions for:
- Loading CSV
- Saving cleaned data locally
- Computing z-scores for outlier detection


In [41]:
# Cell 1 — Setup
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
from datetime import datetime

plt.rcParams["figure.figsize"] = (12,5)
sns.set(style="whitegrid")

# Helper functions
def load_csv(path):
    return pd.read_csv(path, parse_dates=["Timestamp"])

def save_clean(df, out_path="../data/benin-malanville.csv"):
    df.to_csv(out_path, index=False)
    print(f"Saved cleaned dataset to: {out_path}")

def zscore_df(df, cols):
    return df[cols].apply(lambda col: zscore(col.fillna(col.median())))


## 1. Load Dataset & Initial Profiling
- Load Benin solar dataset
- Inspect shape, head, dtypes
- Summary statistics & missing-value report


In [42]:
DATA_PATH = "../data/benin-malanville.csv"
df = load_csv(DATA_PATH)

print("Shape:", df.shape)
display(df.head())
display(df.dtypes)

# Summary & missing report
display(df.describe(include="all"))
missing_counts = df.isna().sum()
missing_pct = (missing_counts / len(df)) * 100
print("Columns with >5% missing values:")
display(missing_pct[missing_pct > 5])


Shape: (525600, 19)


,Timestamp,GHI,DNI,DHI,ModA,ModB,Tamb,RH,WS,WSgust,WSstdev,WD,WDstdev,BP,Cleaning,Precipitation,TModA,TModB,Comments
0,2021-08-09 00:01:00,-1.2,-0.2,-1.1,0.0,0.0,26.2,93.4,0.0,0.4,0.1,122.1,0.0,998,0,0.0,26.3,26.2,NaN
1,2021-08-09 00:02:00,-1.1,-0.2,-1.1,0.0,0.0,26.2,93.6,0.0,0.0,0.0,0.0,0.0,998,0,0.0,26.3,26.2,NaN
2,2021-08-09 00:03:00,-1.1,-0.2,-1.1,0.0,0.0,26.2,93.7,0.3,1.1,0.5,124.6,1.5,997,0,0.0,26.4,26.2,NaN
3,2021-08-09 00:04:00,-1.1,-0.1,-1.0,0.0,0.0,26.2,93.3,0.2,0.7,0.4,120.3,1.3,997,0,0.0,26.4,26.3,NaN
4,2021-08-09 00:05:00,-1.0,-0.1,-1.0,0.0,0.0,26.2,93.3,0.1,0.7,0.3,113.2,1.0,997,0,0.0,26.4,26.3,NaN


Timestamp        datetime64[ns]
GHI                     float64
DNI                     float64
DHI                     float64
ModA                    float64
ModB                    float64
Tamb                    float64
RH                      float64
WS                      float64
WSgust                  float64
WSstdev                 float64
WD                      float64
WDstdev                 float64
BP                        int64
Cleaning                  int64
Precipitation           float64
TModA                   float64
TModB                   float64
Comments                float64
dtype: object

,Timestamp,GHI,DNI,DHI,ModA,ModB,Tamb,RH,WS,WSgust,WSstdev,WD,WDstdev,BP,Cleaning,Precipitation,TModA,TModB,Comments
count,525600,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,0.0
mean,2022-02-07 12:00:30.000000512,240.559452,167.187516,115.358961,236.589496,228.883576,28.179683,54.487969,2.121113,2.809195,0.473390,153.435172,8.582407,994.197199,0.000923,0.001905,35.246026,32.471736,NaN
min,2021-08-09 00:01:00,-12.900000,-7.800000,-12.600000,0.000000,0.000000,11.000000,2.100000,0.000000,0.000000,0.000000,0.000000,0.000000,985.000000,0.000000,0.000000,9.000000,8.100000,NaN
25%,2021-11-08 06:00:45,-2.000000,-0.500000,-2.100000,0.000000,0.000000,24.200000,28.800000,1.000000,1.300000,0.400000,59.000000,3.700000,993.000000,0.000000,0.000000,24.200000,23.600000,NaN
50%,2022-02-07 12:00:30,1.800000,-0.100000,1.600000,4.500000,4.300000,28.000000,55.100000,1.900000,2.600000,0.500000,181.000000,8.600000,994.000000,0.000000,0.000000,30.000000,28.900000,NaN
75%,2022-05-09 18:00:15,483.400000,314.200000,216.300000,463.700000,447.900000,32.300000,80.100000,3.100000,4.100000,0.600000,235.100000,12.300000,996.000000,0.000000,0.000000,46.900000,41.500000,NaN
max,2022-08-09 00:00:00,1413.000000,952.300000,759.200000,1342.300000,1342.300000,43.800000,100.000000,19.500000,26.600000,4.200000,360.000000,99.400000,1003.000000,1.000000,2.500000,81.000000,72.500000,NaN
std,NaN,331.131327,261.710501,158.691074,326.894859,316.536515,5.924297,28.073069,1.603466,2.029120,0.273395,102.332842,6.385864,2.474993,0.030363,0.037115,14.807258,12.348743,NaN


Columns with >5% missing values:


Comments    100.0
dtype: float64

## 2. Clean Column Names & Parse Timestamp
- Standardize column names
- Ensure Timestamp is datetime type
- Sort by Timestamp
- Add derived columns for analysis: date, year, month, hour


In [43]:
df.columns = [c.strip() for c in df.columns]

if not np.issubdtype(df["Timestamp"].dtype, np.datetime64):
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")

df = df.sort_values("Timestamp").reset_index(drop=True)

df["date"] = df["Timestamp"].dt.date
df["year"] = df["Timestamp"].dt.year
df["month"] = df["Timestamp"].dt.month
df["hour"] = df["Timestamp"].dt.hour

display(df[["Timestamp","date","year","month","hour"]].head())


,Timestamp,date,year,month,hour
0,2021-08-09 00:01:00,2021-08-09,2021,8,0
1,2021-08-09 00:02:00,2021-08-09,2021,8,0
2,2021-08-09 00:03:00,2021-08-09,2021,8,0
3,2021-08-09 00:04:00,2021-08-09,2021,8,0
4,2021-08-09 00:05:00,2021-08-09,2021,8,0


## 3. Summary Statistics & Missing-Value Report
- Compute numeric describe
- Columns with missing values >0%
- Save missing-value report locally


In [44]:
numeric = df.select_dtypes(include=[np.number]).columns.tolist()
display(df[numeric].describe().T)

missing_report = pd.DataFrame({
    "missing_count": df.isna().sum(),
    "missing_pct": (df.isna().sum()/len(df))*100
}).sort_values("missing_pct", ascending=False)

display(missing_report[missing_report["missing_pct"] > 0])
missing_report.to_csv("../data/benin_missing_report.csv", index=True)


,count,mean,std,min,25%,50%,75%,max
GHI,525600.0,240.559452,331.131327,-12.9,-2.00,1.8,483.40,1413.0
DNI,525600.0,167.187516,261.710501,-7.8,-0.50,-0.1,314.20,952.3
DHI,525600.0,115.358961,158.691074,-12.6,-2.10,1.6,216.30,759.2
ModA,525600.0,236.589496,326.894859,0.0,0.00,4.5,463.70,1342.3
ModB,525600.0,228.883576,316.536515,0.0,0.00,4.3,447.90,1342.3
Tamb,525600.0,28.179683,5.924297,11.0,24.20,28.0,32.30,43.8
RH,525600.0,54.487969,28.073069,2.1,28.80,55.1,80.10,100.0
WS,525600.0,2.121113,1.603466,0.0,1.00,1.9,3.10,19.5
WSgust,525600.0,2.809195,2.029120,0.0,1.30,2.6,4.10,26.6
WSstdev,525600.0,0.473390,0.273395,0.0,0.40,0.5,0.60,4.2


,missing_count,missing_pct
Comments,525600,100.0


## 4. Outlier Detection
- Compute Z-scores for: GHI, DNI, DHI, ModA, ModB, WS, WSgust
- Flag rows with |Z| > 3


In [45]:
outlier_cols = ["GHI","DNI","DHI","ModA","ModB","WS","WSgust"]
outlier_cols = [c for c in outlier_cols if c in df.columns]

z_df = zscore_df(df, outlier_cols)
outlier_mask = (z_df.abs() > 3).any(axis=1)
outliers = df[outlier_mask]

print("Number of outlier rows detected:", outliers.shape[0])
display(outliers.head(10))


Number of outlier rows detected: 7740


,Timestamp,GHI,DNI,DHI,ModA,ModB,Tamb,RH,WS,WSgust,...,BP,Cleaning,Precipitation,TModA,TModB,Comments,date,year,month,hour
670,2021-08-09 11:11:00,836.0,235.0,610.6,778.8,783.8,30.3,68.2,3.2,4.1,...,999,0,0.0,63.0,52.7,NaN,2021-08-09,2021,8,11
671,2021-08-09 11:12:00,1274.0,698.8,615.2,1210.3,1210.3,30.1,69.6,3.4,4.1,...,999,0,0.0,63.7,53.1,NaN,2021-08-09,2021,8,11
672,2021-08-09 11:13:00,938.0,340.1,612.8,891.1,891.1,30.4,68.4,3.4,4.6,...,999,0,0.0,65.8,54.3,NaN,2021-08-09,2021,8,11
673,2021-08-09 11:14:00,718.5,126.8,593.2,682.6,682.6,30.6,68.2,4.7,5.6,...,999,0,0.0,65.4,54.2,NaN,2021-08-09,2021,8,11
674,2021-08-09 11:15:00,1349.0,771.8,618.0,1281.5,1281.5,30.9,67.1,4.0,4.6,...,999,0,0.0,65.0,53.5,NaN,2021-08-09,2021,8,11
675,2021-08-09 11:16:00,792.6,169.5,625.4,753.0,753.0,31.1,66.7,3.6,4.4,...,999,0,0.0,65.8,53.6,NaN,2021-08-09,2021,8,11
676,2021-08-09 11:17:00,1334.0,751.9,620.0,1267.3,1267.3,31.1,67.0,2.7,3.6,...,999,0,0.0,66.0,53.9,NaN,2021-08-09,2021,8,11
677,2021-08-09 11:18:00,943.0,351.5,602.8,846.0,850.0,31.0,67.9,3.1,4.1,...,999,0,0.0,67.8,55.1,NaN,2021-08-09,2021,8,11
717,2021-08-09 11:58:00,758.9,149.2,601.2,709.9,713.8,30.1,69.2,3.1,3.9,...,999,0,0.0,55.6,47.9,NaN,2021-08-09,2021,8,11
718,2021-08-09 11:59:00,632.8,23.0,603.3,587.0,591.0,30.2,69.3,3.0,4.1,...,999,0,0.0,56.4,48.4,NaN,2021-08-09,2021,8,11


## 5. Cleaning Strategy: Impute & Winsorize
- Impute missing numeric values with median
- Winsorize extreme values to 1st/99th percentile


In [ ]:
clean_df = df.copy()
num_cols = numeric
# Note: You need to ensure 'outlier_cols' is defined before this section runs.

# 1. Impute Missing Numeric Data Safely
for c in num_cols:
    # Check 1: If the column has ANY missing values
    if clean_df[c].isna().any():
        
        # Check 2 (Fixes RuntimeWarning): Only calculate median if there are non-NaN values
        if clean_df[c].notna().any():
            median_val = clean_df[c].median()
            
            # Safe Imputation (Fixes FutureWarning): Use .loc to assign result back
            clean_df.loc[:, c] = clean_df.loc[:, c].fillna(median_val)
        else:
            # Handle cases where the whole column is NaN (e.g., fill with 0 or skip)
            print(f"Warning: Column '{c}' is entirely NaN and was skipped for median imputation.")


# 2. Outlier Handling (Clipping) Safely
for c in outlier_cols:
    low = clean_df[c].quantile(0.01)
    high = clean_df[c].quantile(0.99)
    
    # Safe Clipping (Fixes FutureWarning): Use .loc to assign clipped values back
    clean_df.loc[:, c] = clean_df.loc[:, c].clip(lower=low, upper=high)

# Final steps
z_df_post = zscore_df(clean_df, outlier_cols)
print("Outliers after cleaning:", (z_df_post.abs() > 3).any(axis=1).sum())

C:\Users\yoga\Desktop\10academy\solar-challenge-week0\.venv\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\yoga\AppData\Local\Temp\ipykernel_15552\1660888197.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  clean_df[c].fillna(clean_df[c].median(), inplace=True)


Outliers after cleaning: 0
